In [2]:
#Libaries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from random import shuffle
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from PIL import Image, ImageDraw
import cv2

In [1]:
#Chemin des dossiers avec les données
path = "/home/bbuchi/Bureau/CNN/data"
path_C2SR = path+"/C2SR"
path_TBSR = path+"/TBSR"

#Taille des images (40,30)
width = 40
height = 30

In [4]:
#Pour la création d'une image
def to_img(nom, label, data) :
    w, h = width, height
    print(h)
    d = np.zeros((h, w), dtype=np.uint8)
    for i in range(0,h-1):
        d[i] = data[i]*int(255/3)
    img = Image.fromarray(d)
    imgG = Image.fromarray(np.rot90(d))
    imgD = Image.fromarray(np.fliplr(np.rot90(d)))
    imgI = Image.fromarray(np.flipud(d))
    img.save("bdd/"+label+"/"+nom[:-4]+'.bmp')
    #imgG.save("bdd/"+label+"/"+nom[:-4]+'_G.bmp')
    #imgD.save("bdd/"+label+"/"+nom[:-4]+'_D.bmp')
    #imgI.save("bdd/"+label+"/"+nom[:-4]+'I.bmp')
    img.show()

#Créer les images à partir des fichiers CSV  
def convert_all_img():
    pd.set_option("display.max_rows", None, "display.max_columns", None)    
    for img in os.listdir(path_C2SR) :
            temp = pd.read_csv(path_C2SR+"/"+img,header=None);
            temp = np.array(temp)
            to_img(img,"C2SR",temp)
    for img in os.listdir(path_TBSR) :
            temp = pd.read_csv(path_TBSR+"/"+img,header=None);
            temp = np.array(temp)
            to_img(img,"TBSR",temp)

print("DONE-")

DONE-


In [5]:
def create_data():
    data = []
    for img in os.listdir(path_C2SR) :
        path = path_C2SR+"/"+img
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        data.append([np.array(img), np.array([1,0])])
    for img in os.listdir(path_TBSR) :
        path = path_TBSR+"/"+img
        #print(img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        data.append([np.array(img), np.array([0,1])])
    shuffle(data);
    #data = np.array(data,dtype=object)
    np.save("data.npy",data)
    return data

In [7]:
data = create_data()
taille = len(data)
taille_10p = int(taille*0.1)#10% des données

train_test = data[:-taille_10p]
train = train_test[:-taille_10p]
test = train_test[-taille_10p:]
valid = data[-taille_10p:]

w_ = train[0][0][0].size
h_ = int(train[0][0].size/w_)

train_X = np.array([i[0] for i in train]).reshape(-1,w_,h_,1)
train_X=train_X/255
train_y = np.array([i[1] for i in train])

test_X = np.array([i[0] for i in test]).reshape(-1,w_,h_,1)
test_X=test_X/255
test_y = np.array([i[1] for i in test])

valid_X = np.array([i[0] for i in test]).reshape(-1,w_,h_,1)
valid_X=valid_X/255
valid_y = np.array([i[1] for i in test])

y_ = []
for i in train_y:
    if i[0] == 1 :
        y_.append(0)
    else :
        y_.append(1)
y_ = np.array(y_)
train_y = y_

y_ = []
for i in test_y:
    if i[0] == 1 :
        y_.append(0)
    else :
        y_.append(1)
y_ = np.array(y_)
test_y = y_

y_ = []
for i in valid_y:
    if i[0] == 1 :
        y_.append(0)
    else :
        y_.append(1)
y_ = np.array(y_)
valid_y = y_


print("DONE-")

DONE-


In [8]:
#Le CNN
k1=3
k2=5

model = models.Sequential()
model.add(layers.Conv2D(filters=10, kernel_size=(k1,k1), activation="relu", input_shape=(width,height,1)))
model.add(layers.AveragePooling2D())
model.add(layers.Conv2D(filters=16, kernel_size=(k1,k1), activation="relu"))
model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(120, activation="relu"))
model.add(layers.Dense(84, activation="relu"))
model.add(layers.Dense(2, activation="softmax"))
model.summary()

modelBig = models.Sequential()
modelBig.add(layers.Conv2D(filters=10, kernel_size=(k2,k2), activation="relu", input_shape=(width,height,1)))
modelBig.add(layers.AveragePooling2D())
modelBig.add(layers.Conv2D(filters=16, kernel_size=(k2,k2), activation="relu"))
modelBig.add(layers.AveragePooling2D())
modelBig.add(layers.Flatten())
modelBig.add(layers.Dense(120, activation="relu"))
modelBig.add(layers.Dense(84, activation="relu"))
modelBig.add(layers.Dense(2, activation="softmax"))
modelBig.summary()

print("DONE")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 28, 10)        100       
_________________________________________________________________
average_pooling2d (AveragePo (None, 19, 14, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 12, 16)        1456      
_________________________________________________________________
average_pooling2d_1 (Average (None, 8, 6, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               92280     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [116]:
#Entrainer le réseau de neurones
from tensorflow import keras
opt=keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_X, train_y, epochs=80, validation_data=(test_X,test_y))

#modelBig.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #metrics=['accuracy'])
#historyBig = modelBig.fit(train_X, train_y, epochs=25, validation_data=(test_X,test_y))

Epoch 1/80
8/8 [==============================] - 0s 60ms/step - loss: 0.6894 - accuracy: 0.5248 - val_loss: 0.6833 - val_accuracy: 0.5333
Epoch 2/80
8/8 [==============================] - 0s 33ms/step - loss: 0.6735 - accuracy: 0.5744 - val_loss: 0.6657 - val_accuracy: 0.5333
Epoch 3/80
8/8 [==============================] - 0s 50ms/step - loss: 0.6595 - accuracy: 0.5744 - val_loss: 0.6545 - val_accuracy: 0.5333
Epoch 4/80
8/8 [==============================] - 0s 51ms/step - loss: 0.6539 - accuracy: 0.5744 - val_loss: 0.6419 - val_accuracy: 0.5333
Epoch 5/80
8/8 [==============================] - 0s 43ms/step - loss: 0.6459 - accuracy: 0.6033 - val_loss: 0.6293 - val_accuracy: 0.6667
Epoch 6/80
8/8 [==============================] - 0s 20ms/step - loss: 0.6485 - accuracy: 0.6198 - val_loss: 0.6210 - val_accuracy: 0.6667
Epoch 7/80
8/8 [==============================] - 0s 14ms/step - loss: 0.6422 - accuracy: 0.6198 - val_loss: 0.6196 - val_accuracy: 0.6667
Epoch 8/80
8/8 [===========

In [148]:
#Affiche les résultats pour les données qui n'ont pas servi à l'entrainement du systeme
#shfl()
#predictions = model.predict_classes(valid_X)
#print(valid_y)
scores = model.evaluate(valid_X,valid_y)
#scoresBig = modelBig.evaluate(valid_X,valid_y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#print("\n%s: %.2f%%" % (modelBig.metrics_names[1], scoresBig[1]*100))
print("=================================")

predictions = model.predict_classes(valid_X)
#predictionsBig = modelBig.predict_classes(valid_X)


ind = 1
ok1=0
ok0=0
takes = ok1;
for i, j in zip(predictions , valid_y):
    #print( "{} <= {} // {} + {}:{}".format(j,i,i==j,ok0,ok1))
    if j==1:
        ok1=ok1+1
        takes = "1 +"+str(ok1)
    else:
        ok0=ok0+1
        takes = "0 +"+str(ok0)
    if i!=j :
        print(str(takes))
    ind = ind+1
print("0 ("+str(ok0)+")   &&   1 ("+str(ok1)+")")

#print("-------------")

ind = 92
#for i, j in zip(predictionsBig , valid_y):
#    print( "{}| {} <= {} // {}".format(ind,j,i,i==j))
#    ind = ind+1

1/1 [==============================] - 0s 1ms/step - loss: 0.3485 - accuracy: 0.9667

accuracy: 96.67%
1 +7
0 (13)   &&   1 (17)
